In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/Users/perceval/Development/jupytanno/")

from jupytanno import App, kernel_only, frontend_only, produce, chain_map
from jupytanno.immutable import commit
from jupytanno.python_to_javascript import transkrypt_class
from jupytanno.brat import load_from_brat
import logging

from collections import Counter
import re

colors = [
"rgb(255,200,206)",
"rgb(210,236,247)",
"rgb(211,242,206)",
"rgb(242,242,206)",
"rgb(231,210,247)",
"rgb(252,215,216)",
"rgb(251,243,219)",
"rgb(250,231,212)",
"rgb(250,212,229)",
]

class BRATApp(App):
    def __init__(self, path=None):
        super().__init__()
        
        state = {
            "text": "",
            "spans": [],
            "docs": [],
            "styles": {},
            "next_span_id": 1,
            "mouse_selection": [],
            "attributes": [],
        }
        self.state = state
        self.label_config = {
            "neg": ("#ff5b5b", "n"),
            "quad": ("#0f8edc", "q"),
        }
        
        if path is not None:
            self.dataset = list(load_from_brat(path))
            labels = [label for label, count in sorted(Counter(e["label"] for d in self.dataset for e in d["entities"]).items(), key=lambda x: -x[1])]
            self.state['docs'] = [{"id": doc["doc_id"]} for doc in self.dataset]
            self.state["styles"] = {
                label: {"color": self.label_config.get(label, (color,))[0], "alpha": 0.8}
                for label, color in zip(labels, colors)
            }
            self.state["attributes"] = sorted(set(a['label'] for d in self.dataset for e in d['entities'] for a in e['attributes']))
            self.change_doc(self.dataset[0]['doc_id'])
        
    def on_state_change(self, state, old_state):
        #logs.append(state["spans"] is not old_state["spans"])
        pass
    
    def select_editor_state(self, state, editor_id):
        if editor_id == "my-editor":
            return dict(
                text=state["text"],
                spans=[{
                            "begin": span["begin"],
                            "end": span["end"],
                            "label": [label for label in span["label"] if label in state["styles"]][0],
                            "style": [label for label in span["label"] if label in state["styles"]][0],
                            "id": span["id"],
                            "highlighted": span["highlighted"],
                            "selected": span["selected"],
                        } for span in state["spans"]],
                styles=state["styles"],
                mouse_selection=state["mouse_selection"],
                buttons=[],
            )
        elif editor_id == "docs-table":
            return dict(
                rows=[{
                    "doc_id": {"key": doc["id"], "text": doc["id"]},
                    "key": doc["id"],
                    "highlighted": False,
                } for doc in state["docs"]],
                rowKey="key",
                columns=[
                    {
                        "name": "doc_id",
                        "type": "hyperlink",
                    },
                ],
                selectedRows=[],
            )
        elif editor_id == "mentions-table":
            res = dict(
                rows=[{
                    "id": span["id"],
                    "highlighted": span["highlighted"],
                    "visible": True,
                    "mention": {
                        "text": state["text"][span["begin"]:span["end"]],
                        "key": span["id"],
                    },
                    "labels": chain_list(span["label"], [span['attributes'][a] for a in state['attributes'] if a in span['attributes']]),
                    "custom_link": span["custom_link"],
                } for span in state["spans"]],
                rowKey="id",
                columns=[
                    {
                        "name": "mention",
                        "type": "hyperlink",
                    },
                    {
                        "name": "labels",
                        "type": "multi-text",
                        "suggestions": [],
                    },
                    {
                        "name": "custom_link",
                        "type": "hyperlink",
                        "readonly": False,
                        "suggestions": [
                            {"text": state["text"][span["begin"]:span["end"]], "key": span["id"]}
                            for span in state["spans"]
                        ],
                    },
                ],
                selectedRows=[span["id"] for i, span in enumerate(state["spans"]) if span["selected"]],
            )
            #res['columns'].extend([
            #    {
            #        "name": name,
            #        "type": "text",
            #    } for name in state['attributes']
            #])
            #print(res['columns'])
            return res
        raise Exception()
        
    @produce
    def handle_mouse_select(self, editor_id, modkeys, spans):
        print(spans)
        if "Shift" in modkeys:
            self.state["mouse_selection"].extend(spans)
        else:
            self.state["mouse_selection"] = spans
    
    @produce
    def handle_button_press(self, editor_id, button_idx, selections):
        button_presses.append((editor_id, button_idx, selections))
    
    @produce
    @frontend_only
    def handle_enter_span(self, editor_id, span_id, modkeys):
        for span in self.state["spans"]:
            if span['id'] == span_id:
                span['highlighted'] = True
            
    @produce
    @frontend_only
    def handle_leave_span(self, editor_id, span_id, modkeys):
        for span in self.state["spans"]:
            if span['id'] == span_id:
                span['highlighted'] = False
    
    @produce
    def handle_key_press(self, editor_id, key, modkeys, spans):
        def has_overlap(x, y):
            return not (x['end'] <= y['begin'] or y['end'] <= x['begin'])
        text = self.state["text"]
        next_span_id = self.state["next_span_id"]
        if key == " ":
            first_selection = self.state["mouse_selection"][0]
            term = self.state["text"][first_selection["begin"]:first_selection["end"]]
            next_occurence = self.state["text"].find(term, self.state["mouse_selection"][len(self.state["mouse_selection"])-1]["end"])
            if next_occurence > 0:
                self.state["mouse_selection"].append({"begin": next_occurence, "end": next_occurence + len(term)})
        if key == "Backspace":
            self.state["spans"] = [
                span
                for span in self.state["spans"]
                if not any(has_overlap(span, mouse_span) for mouse_span in spans)
            ]
        elif len(spans):
            new_spans = []
            for span in spans:
                for label, (color, shortcut) in self.label_config.items():
                    if key == shortcut:
                        new_spans.append({
                            "begin": span["begin"],
                            "end": span["end"],
                            "label": [label],
                            "id": "T{}".format(next_span_id),
                            "highlighted": False,
                            "selected": False,
                            "custom_link": "",
                        })
                        next_span_id += 1
                        added_spans = True
                if key == " ":
                    pass
                    # Do something
                    #added_spans = True
            if not len(new_spans):
                return
            self.state["spans"].extend(new_spans)
        self.state["mouse_selection"] = []
        self.state["next_span_id"] = next_span_id
            
    @produce
    def handle_click_span(self, editor_id, span_id, modkeys):
        if "Shift" in modkeys:
            for span in self.state["spans"]:
                if span['id'] == span_id:
                    span['selected'] = not span['selected']
        
    @kernel_only
    @produce
    def change_doc(self, key):
        doc = next(doc for doc in self.dataset if doc["doc_id"] == key)
        self.state["text"] = doc["text"]
        self.state["doc_id"] = doc["doc_id"]
        self.state["spans"] = [
            {
                "begin": s["begin"],
                "end": s["end"],
                "label": [e["label"]],
                "highlighted": False,
                "selected": False,
                "id": e["entity_id"],
                "key": e["entity_id"],
                "custom_link": "",
                "attributes": {
                    a['label']: a['value']
                    for a in e['attributes']
                }
            }
            for e in doc["entities"]
            for s in e["fragments"]
        ]
        self.state["mouse_selection"] = []
        self.state["next_span_id"] = 1
    
    def handle_click_cell_content(self, editor_id, key):
        if editor_id == "docs-table":
            print("Changing doc")
            self.change_doc(key)
        else:
            self.scroll_to_span("my-editor", key)
    
    @produce
    def handle_select_rows(self, editor_id, span_ids):
        for span in self.state["spans"]:
            span['selected'] = span['id'] in span_ids
            
    @produce
    def handle_cell_change(self, editor_id, row_idx, col, value):
        console.log("CELL CHANGE", editor_id, row_idx, col, value)
        if col == "label":
            self.state["spans"][row_idx]["label"] = value
        if col == "custom_link":
            self.state["spans"][row_idx]["custom_link"] = value
    
    # @kernel_only
    # def add_auto_span(self, editor_id, begin, end):
    #     mention_text = self.get_state()["editors"][editor_id]["text"][begin:end]
    #     #label = self.ml.predict(mention_text)
    #     label = mysuper_complicated_function(mention_text)
    #     self.add_one_span(editor_id, begin, end, label, label)
        
import time
def mysuper_complicated_function(mention_text):
    # time.sleep(3)
    return "temp" if len(mention_text) > 10 else "neg"

try: app.set_class(BRATApp)
except: pass

In [49]:
app = BRATApp("/Users/perceval/Development/data/resources/brat/deft-2020-t3/t3-appr/")

In [18]:
app.change_doc("filepdf-216-cas")

In [50]:
app.span_editor("my-editor")

In [51]:
app.span_editor("my-editor")

In [52]:
app.table_editor(name="mentions-table")

In [53]:
app.table_editor(name="docs-table")

In [4]:
buttons=[
            #    {"type": 'button', "label": "⌫", "color": 'white', "key": "delete"},
            #    {"type": 'spacer'},
            #    {"type": 'button', "label": "Lateralité", "secondary": self.label_config["lat"][1], "color": self.label_config["lat"][0], "key": "lat"},
            #    {"type": 'button', "label": "ACR", "secondary": self.label_config["acr"][1], "color": self.label_config["acr"][0], "key": "acr"},
            #    {"type": 'button', "label": "Negation", "secondary": self.label_config["neg"][1], "color": self.label_config["neg"][0], "key": "neg"},
            #    {"type": 'button', "label": "Quadrant", "secondary": self.label_config["quad"][1], "color": self.label_config["quad"][0], "key": "quad"},
            #    {"type": 'button', "label": "Cible", "secondary": self.label_config["target"][1], "color": self.label_config["target"][0], "key": "target"},
            #    {"type": 'button', "label": "Mesure", "secondary": self.label_config["mesure"][1], "color": self.label_config["mesure"][0], "key": "mesure"},
            #    {"type": 'button', "label": "Temporalité", "secondary": self.label_config["temp"][1], "color": self.label_config["temp"][0], "key": "temp"},
            #    *buttons,
            #
]